In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
785,This is one of the worst movies I've seen in a...,negative
251,If you want to be cynical and pedantic you cou...,positive
348,"""Look, I know this may suck right now, but pai...",negative
603,The first 10 minutes of the movie makes fun of...,negative
686,"Firstly,I must admit that it isn't a good movi...",negative


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
# import nltk
# nltk.download('stopwords')

In [6]:
# import nltk
# print(nltk.data.path)

In [7]:
# import nltk
# from nltk.corpus import stopwords

In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
df = normalize_text(df)
df.head()

,review,sentiment
785,one worst movie seen long time story boring di...,negative
251,want cynical pedantic could point opening raf ...,positive
348,look know may suck right now pain temporary fi...,negative
603,first minute movie make fun sequel pg movie re...,negative
686,firstly i must admit good movie and i would ne...,negative


In [10]:
df['sentiment'].value_counts()

sentiment
negative    265
positive    235
Name: count, dtype: int64

In [11]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [12]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
785,one worst movie seen long time story boring di...,0
251,want cynical pedantic could point opening raf ...,1
348,look know may suck right now pain temporary fi...,0
603,first minute movie make fun sequel pg movie re...,0
686,firstly i must admit good movie and i would ne...,0


In [13]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [19]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 11037 stored elements and shape (500, 100)>
  Coords	Values
  (0, 59)	2
  (0, 52)	2
  (0, 72)	1
  (0, 44)	1
  (0, 88)	1
  (0, 78)	1
  (0, 0)	1
  (0, 26)	1
  (0, 41)	1
  (0, 21)	1
  (0, 79)	1
  (0, 93)	1
  (1, 59)	1
  (1, 52)	3
  (1, 78)	1
  (1, 26)	4
  (1, 21)	1
  (1, 90)	1
  (1, 17)	1
  (1, 65)	1
  (1, 75)	1
  (1, 18)	1
  (1, 15)	1
  (1, 24)	2
  (1, 97)	1
  :	:
  (499, 54)	1
  (499, 61)	1
  (499, 13)	2
  (499, 81)	1
  (499, 64)	1
  (499, 40)	1
  (499, 71)	1
  (499, 28)	1
  (499, 39)	2
  (499, 91)	1
  (499, 98)	1
  (499, 22)	2
  (499, 85)	1
  (499, 6)	1
  (499, 86)	1
  (499, 14)	2
  (499, 35)	1
  (499, 67)	2
  (499, 29)	1
  (499, 34)	2
  (499, 38)	3
  (499, 92)	1
  (499, 94)	1
  (499, 25)	1
  (499, 46)	1


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [36]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/alhasannur99/Sentiment-analysis.mlflow')
dagshub.init(repo_owner='alhasannur99', repo_name='Sentiment-analysis', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as alhasannur99

Initialized MLflow to track repo "alhasannur99/Sentiment-analysis"

Repository alhasannur99/Sentiment-analysis initialized!

<Experiment: artifact_location='mlflow-artifacts:/f2e48fc3a1db49c6aeb88e4b4a16a313', creation_time=1756830515425, experiment_id='0', last_update_time=1756830515425, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [37]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025/09/04 15:27:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run shivering-goat-430 at: https://dagshub.com/alhasannur99/Sentiment-analysis.mlflow/#/experiments/0/runs/e2ce8c1140534d8fa4503a77487cd11d
🧪 View experiment at: https://dagshub.com/alhasannur99/Sentiment-analysis.mlflow/#/experiments/0
